## データの取得・確認

In [1]:
import sys
sys.path.append(r'C:/Users/koki5/Dropbox/Jupyter/ScratchML')

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import model_selection
from termcolor import cprint

from common.module.evaluation import score


cancer = datasets.load_breast_cancer()
cancer_data = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
cancer_target = pd.Series(data=cancer.target, name='target')

X_train, X_test, y_train, y_test = model_selection.train_test_split(cancer_data, cancer_target)

cprint('Cancer Data (samples: {}, features: {})'.format(*cancer_data.shape), 'blue', attrs=['bold'])
cprint('Cancer Target (samples: {})'.format(*cancer_target.shape), 'blue', attrs=['bold'])

Cancer Data (samples: 569, features: 30)
Cancer Target (samples: 569)


In [2]:
cancer_data.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [3]:
cancer_target.unique()

array([0, 1], dtype=int64)

## データ前処理

In [4]:
def standardize(X):
    """標準化する.
    
    X: データセット
    """
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)


def add_ones(X):
    """特徴量の0列目に1を加える.
    
    X: データセット
    """
    X['0'] = 1
    return X.sort_index(axis=1, ascending=True)


X_train_std = standardize(X_train)
X_test_std = standardize(X_test)

# バイアス項を追加
X_train_std = add_ones(X_train_std)
X_test_std = add_ones(X_test_std)

X_train_std.head(3)

,0,area error,compactness error,concave points error,concavity error,fractal dimension error,mean area,mean compactness,mean concave points,mean concavity,...,worst area,worst compactness,worst concave points,worst concavity,worst fractal dimension,worst perimeter,worst radius,worst smoothness,worst symmetry,worst texture
169,1,-0.400498,-0.672352,-0.466936,-0.789417,-0.814867,0.046303,-0.513131,-0.336391,-0.805864,...,-0.197775,-0.596460,-0.496601,-1.007166,-1.087824,-0.125743,-0.083943,-0.494034,-0.794926,-0.446976
111,1,-0.486359,0.817553,1.765658,0.567971,0.813268,-0.525619,0.246843,0.219245,0.157030,...,-0.642229,-0.214045,-0.114658,-0.277967,0.019396,-0.574064,-0.639102,-0.192658,-1.075135,-0.042225
127,1,0.886275,0.086541,0.262997,0.238665,-0.433607,1.304531,-0.487286,0.121513,-0.007959,...,1.044461,-0.205311,0.053605,0.187890,-1.027029,1.127257,1.156178,-1.321757,-0.106998,0.000381


## モデルの構築・訓練

In [5]:
class LogisticRegression(object):
    """勾配降下法に基づくロジスティック回帰."""
    
    
    def __init__(self, alpha=0.01, eps=1e-6):
        """
        ---パラメータ-------
        alpha: 学習率
        eps: 収束の目安
        --------------------
        
        ---属性-------------
        _w: パラメータ（重み）
        --------------------
        """
        self.alpha = alpha
        self.eps = eps
        
        self._w = 0
        
        
    def fit(self, X, y):
        """訓練データで学習する.
        
        ---パラメータ-------
        X: 訓練データセット
        y: ターゲット
        --------------------
        
        """
        # パラメータの初期値はランダムに設定する
        theta = np.random.rand(X.shape[1])
        error = self.J(X, y, theta)
        count = 0
        diff = 1
        
        # コスト関数の値が殆ど変動しなくなったら収束とする
        while diff > self.eps:
            # パラメータの更新処理
            grad = (1 / len(y)) * np.dot(self.h(X, theta) - y, X)
            theta = theta - self.alpha * grad
            
            # 収束状況を確認するための処理
            current_error = self.J(X, y, theta)
            diff = error - current_error
            error = current_error
            count += 1
            print('\r【{}回目】Error: {}, Diff: {}, Grad: {}'.format(count, error, diff, grad.sum()), end='')
            
        self._w = theta
        return self
    
    
    def predict(self, X):
        """クラスラベルを予測する.
        
        X: テストデータセット
        """
        return np.where(self.h(X, self._w) >= 0.5, 1, 0)
        
    
    def activate(self, z):
        """ロジスティック関数（活性化関数）.
        
        z: 仮説関数の予測値
        """
        return 1 / (1 + np.exp(-z))
    

    def h(self, X, theta):
        """仮説関数.
        
        X: 訓練データセット
        theta: パラメータ（重み）
        """
        return self.activate(np.dot(X, theta))
    

    def J(self, X, y, theta):
        """目的関数.
        
        X: 訓練データセット
        y: ターゲット
        theta: パラメータ（重み）
        """
        delta = 1e-7 # np.log()に0が混在しないようにする
        return - (1 / len(y)) * (np.sum(y * np.log(self.h(X, theta) + delta) + (1 - y) * np.log(1 - self.h(X, theta) + delta)))

In [6]:
lr = LogisticRegression()
lr.fit(X_train_std, y_train)

【106回目】Error: 1.011361220026064, Diff: 0.026608091480369023, Grad: 8.644549111046585

【204回目】Error: 0.2661351638424891, Diff: 0.0015544563338350526, Grad: 2.0930662680563143

【308回目】Error: 0.18784895878375035, Diff: 0.0003703063347734181, Grad: 0.9860678889098045

【414回目】Error: 0.16132315161580343, Diff: 0.00017467491985770356, Grad: 0.6274674272360268

【521回目】Error: 0.14662524090996953, Diff: 0.00010998643877715986, Grad: 0.4583983331508652

【631回目】Error: 0.1364483384382285, Diff: 7.858525386272852e-05, Grad: 0.36009782391277434

【11345回目】Error: 0.06650796901344012, Diff: 9.99970273890538e-07, Grad: 0.01997338633024684387

## 予測・評価

In [7]:
predict = pd.DataFrame(lr.predict(X_train_std), columns=['TARGET'])
predict.head(10)

,TARGET
0,1
1,1
2,0
3,0
4,1
5,0
6,1
7,0
8,1
9,1


In [8]:
train_score = score(predict, y_train)
test_score = score(pd.DataFrame(lr.predict(X_test_std), columns=['target']), y_test)
cprint('train_score: {}\ntest_score: {}'.format(train_score, test_score), 'red', attrs=['bold'])

train_score: 0.9812206572769953
test_score: 0.9370629370629371
